# Objective: To develop a mathematical understanding of PCA

# Test data

In [ ]:
import numpy as np
from scipy import linalg as LA
X = np.array([
        [0.387,4878, 5.42],
        [0.723,12104,5.25],
        [1,12756,5.52],
        [1.524,6787,3.94],
    ])
X = X - np.mean(X, axis=0)

# Non-Iinear Iterative Partial Least-Squares (NIPALS) algorithm

Steps to compute PCA using NIPALS algorithm

* Step 1: Initialize an arbitrary column vector $\mathbf{t}_{a}$ either randomly or by just copying any column of X. 
* Step 2: Take very column of $\mathbf{X}$, $\mathbf{X_k}$ and regress it onto the $\mathbf{t}_{a}$ vector and store the regression coefficeints as $\mathbf{p}_{ka}$. (Note: This simply means performing an ordinary least squares regression ($y=mx$) with $x=t_{a}$ and $y=X_{k}$ with $m=(\mathbf{x^T}\mathbf{x})^{-1}\mathbf{x^T}\mathbf{y}$). In the current notation we get 
$$p_{ka}=\frac{\mathbf{t_a^T}\mathbf{X}_{k}}{\mathbf{t_a^T}\mathbf{t_a}}$$

Repeat it for each of the columns of $\mathbf{X}$ to get the entire vector $\mathbf{p}_{k}$. This is shown in the illustration
above where each column from $X$ is regressed, one at a time, on $\mathbf{t}_{a}$, to calculate the loading entry, $\mathbf{𝑝}_{ka}$ 

In practice we don’t do this one column at time; we can regress all columns in $X$ in go: $$\mathbf{p_a^T}=\frac{1}{\mathbf{t_a^T}\mathbf{t_a}}.\mathbf{t_a^T}\mathbf{X_a}$$  where $\mathbf{t_a}$ is an $N \times 1$ column vector, and $\mathbf{X}_{a}$ us an $N \times K$ matrix.
* The loading vector $\mathbf{p_a^T}$ won’t have unit length (magnitude) yet. So we simply rescale it to have
magnitude of 1.0: $$\mathbf{p_a^T}=\frac{\mathbf{p_a^T}}{\sqrt{\mathbf{p_a^T}\mathbf{p_a}}}$$
* Step 4: Regress every row in $X$ onto this normalized loadings vector. As illustrated below, in our linear regression the rows in X are our y-variable each time, while the loadings vector is our x-variable. The regression coefficient becomes the score value for that $𝑖^{th}$ row:

$$p_{i,a}=\frac{\mathbf{x}_{i}^{T}\mathbf{p}_{a}}{\mathbf{p}_{a}^{T}\mathbf{p}_{a}}$$
where $x_{i}^{T}$ is a $K \times 1$ column vector. We can combine these $N$ separate least-squares models and
calculate them in one go to get the entire vector, 

$$\mathbf{t}_{a}^{T}=\frac{1}{\mathbf{p}_{a}^{T}\mathbf{p}_{a}}\mathbf{X}\mathbf{p}_{a}^{T}$$ 

where $p_{a}$ is a $K \times 1$ column vector.

* Step 5: Continue looping over steps 2,3,4 until the change in vector $t_{a}$ is below a chosen tolerance
* Step 6: On convergence, the score vector and the loading vectors, $\mathbf{t}_{a}$ and $\mathbf{p}_{a}$ are stored as the $a^{th}$ column in matrix $\mathbf{T}$ and $\mathbf{P}$. We then deflate the $\mathbf{X}$ matrix. This crucial step removes the variability captured in this component ($t_{a}$ and $p_{a}$) from $\mathbf{X}$:

$$E_{a}=X_{a}-t_{a}p_{a}^{T}$$

$$X_{a+1} = E_{a}$$ 

For the first component, $X_{a}$ is just the preprocessed raw data. So we can see that the second component is actually calculated on the residuals $E_{1}$, obtained after extracting the first component. This is called deflation, and nicely shows why each component is orthogonal to the others. Each subsequent component is only seeing variation remaining after removing all the others; there is no possibility that two components can explain the same type of variability. After deflation we go back to step 1 and repeat the entire process for the next component. 

## IMPLEMENTATION IN PYTHON

In [ ]:
def PCA(X,no_components):
    tol = 0.0000001
    it=1000
    obsCount,varCount = X.shape
    Xa = X - np.mean(X, axis = 0) 
    #Xh = X-np.tile(np.mean(X,axis=0).reshape(-1,1).T, obsCount).reshape(4,3)
    T = np.zeros((obsCount,no_components))
    P = np.zeros((varCount,no_components))
    pcvar = np.zeros((varCount,1))
    varTotal = np.sum(np.var(Xa,axis=0,ddof=1))
    currVar = varTotal
    nr=0
    for h in range(no_components):
        th = Xa[:,0].reshape(-1,1)
        ende = False
        while ende != True:
            nr = nr + 1
            ph = np.dot(Xa.T,th)/np.dot(th.T,th)
            ph = ph /np.linalg.norm(ph)
            thnew = np.dot(Xa,ph)/np.dot(ph.T,ph)
            prec = np.dot((thnew-th).T,(thnew-th))
            th = thnew
            if prec <= (tol*tol):
                ende = True
            elif it <=nr:
                ende = True
                print("Iternation stops without convergence")
        Ea = Xa - np.dot(th,ph.T)
        Xa = Ea    
        T[:,h] = th.flatten()
        P[:,h] = ph.flatten()
        oldVar = currVar
        currVar = np.sum(np.var(Xa,axis=0,ddof=1))
        pcvar[h] = (oldVar - currVar) / varTotal
        nr = 0
    return T,P,pcvar      

## Advantages of the NIPALS algorithm
* The NIPALS algorithm computes one component at a time. The first component computed is
equivalent to the t1 and p1 vectors that would have been found from an eigenvalue or singular value
decomposition.
* The algorithm can handle missing data in X.
* The algorithm always converges, but the convergence can sometimes be slow.
* It is also known as the Power algorithm to calculate eigenvectors and eigenvalues.
* It works well for very large data sets.
* It is used by most software packages, especially those that handle missing data.
* Of interest: it is well known that Google used this algorithm for the early versions of their search engine, called PageRank148.

In [ ]:
no_components=3
T,P,pcvar = PCA(X,no_components)
print("T (Scores)")
print(T)
print(" ")
print("P (Loadings)")
print(P)
print(np.sqrt(pcvar)/np.sum(np.sqrt(pcvar)))

# SVD

In [ ]:
from numpy.linalg import svd 
U, S, PTrans = svd(X, full_matrices=False)
Sigma = np.diag(S)
T=np.dot(U,Sigma)
P=PTrans.T

print("T (Scores)")
print(T)
print(" ")
print("P (Loadings)")
print(P)
print("Sigma (Variance)")
print(S)

# SKLEARN PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()  
T=pca.fit_transform(X)
Prans=pca.components_ #eigen vectors.T
latent = pca.explained_variance_
explained = pca.explained_variance_ratio_
P=PTrans.T
S=pca.singular_values_
Sigma=np.diag(S)
print("T (Scores)")
print(T)
print(" ")
print("P (Loadings)")
print(P)
print("Sigma (Variance)")
print(S)
#print(pca.singular_values_/np.sqrt(3))

In [ ]:
pca.explained_variance_ratio_

In [ ]:
explained_variance_2 = (S ** 2) / 4
explained_variance_ratio_2 = (explained_variance_2 / explained_variance_2.sum())
print(explained_variance_ratio_2)

# Eigenvalue decomposition approach

Recall that the latent variable directions (the loading vectors) were oriented so that the variance of the
scores in that direction were maximal. We can cast this as an optimization problem. For the first
component: $$max (\phi)=\mathbf{t_1^T}\mathbf{t_1}=\mathbf{p_1^T} \mathbf{X^T}\mathbf{Xp_1}$$
such that $$\mathbf{p_1^T p_1}=1$$.

This is equivalent to $$max (\phi)=\mathbf{p_1^T} \mathbf{X^T Xp_1}-\lambda(\mathbf{p_1^T}\mathbf{p_1}-1)$$ 

because we can move the constraint into the objective function with a Lagrange multiplier, $\lambda$. The maximum value must occur when the partial derivatives with respect to $\mathbf{p_1}$, 

our search variable, are zero: $$\frac{\partial \phi}{\partial p_1}= \frac{\partial (\mathbf{p_1^T X^T Xp_1}-\lambda(\mathbf{p}_{1}^{T}\mathbf{p}_{1}-1))}{\partial \mathbf{p}_1}=0$$

$$2\mathbf{X^T X p_1}-2\lambda_1\mathbf{p_1}=0$$

$$(\mathbf{X^TX}-\lambda_1\mathbf{I})\mathbf{p_1}=0$$

$$\mathbf{X^T Xp_1}=\lambda_{1}\mathbf{p_1}$$

which is just the eigenvalue equation, indicating that $\mathbf{p_1}$ is the eigenvector of $\mathbf{X^T X}$ and $\lambda_1$ is the eigenvalue. One can show that $\lambda_1=\mathbf{t_1^T t_1}$, which is proportional to the variance of the first component. In a similar manner we can calculate the second eigenvalue, but this time we add the additional constraint that $\mathbf{p}_1 \perp \mathbf{p}_2$. Writing out this objective function and taking partial derivatives leads to showing that 

$$\mathbf{X^{T}Xp_{2}} = \lambda_{2} \mathbf{p_2}$$

From this we learn that:
* The loadings are the eigenvectors of $\mathbf{X^TX}$.
* Sorting the eigenvalues in order from largest to smallest gives the order of the corresponding eigenvectors, the loadings.
* We know from the theory of eigenvalues that if there are distinct eigenvalues, then their eigenvectors are linearly independent (orthogonal).
* We also know the eigenvalues of $\mathbf{X^TX}$ must be real values and positive; this matches with the interpretation that the eigenvalues are proportional to the variance of each score vector.
* Also, the sum of the eigenvalues must add up to sum of the diagonal entries of $\mathbf{X^TX}$, which represents of the total variance of the $\mathbf{X}$ matrix, if all eigenvectors are extracted. So plotting the eigenvalues is equivalent to showing the proportion of variance explained in X by each component. This is not necessarily a good way to judge the number of components to use, but it is a rough guide: use a Pareto plot of the eigenvalues (though in the context of eigenvalue problems, this plot is called a scree plot).

In [ ]:
cov = np.cov(X, rowvar = False)
evals , P = LA.eigh(cov)
idx = np.argsort(evals)[::-1]
P = P[:,idx]
evals = evals[idx]
T = np.dot(X, P) 
Sigma=LA.norm(T,axis=0)
print("T (Scores)")
print(T)
print("P (Loadings)")
print(P)
print("Sigma (Variance)")
print(Sigma)

## Task 1: Test if the loading vectors are orthogonal and orthonormal or not

In [ ]:
""" Your code here """


""" ---         ---"""

## Task 2: Test if the scores vectors are orthogonal and orthonormal or not

In [ ]:
""" Your code here """


""" ---         ---"""

## Task 3: Add more columns to the original data matrix by: 
* Make some of the columns to be the linear combination of others
* Duplicate some columns
* Add noise as some columns 
* Add a few columns of categorical values

Then apply PCA to the dataset and report your findings here

In [ ]:
""" Your code here """


""" ---         ---"""

# Task 4: Find the PCA of the original data without mean centering and report the findings

In [ ]:
""" Your code here """


""" ---         ---"""